


# MINOR PROJECT 

                                                        

#Sentiment Analysis



#Importing the LIABRARIES which were needed to work on this project  

Python library,visualisation library,NLTK library,regular expression

In [189]:
from googleapiclient.discovery import build

#this NLTK package is used to perform cleaning of data
#word_tonkenize is used to sperate  each words in the perform of list
#sent_tonkenize is used to   seprate  each sentence in the sentnece  in the form of list

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation #punctutaion are . , all this
from string import punctuation  # punctutaion are . , all this

#stopwords package is used to remove the unneccessary words from the list 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.collocations import BigramCollocationFinder
import nltk
from nltk.corpus import stopwords

#the wordcloud package in the python is used to generate the wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import unicodedata
import re
import html

#data visualisation library
import numpy as np
from PIL import Image

#this pandas library help to perform the dataframe
import pandas as pd
import itertools 
from collections import OrderedDict
import collections

#this perform the polarity of the sentiments
import textblob
from textblob import TextBlob

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline 
try:
    nltk.download('stopwords')
except:
    pass


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Enter your APIs key 

In [190]:
api_key = 'your api key'



#Making class object to work on youtube api
#EXTRACING DATA FROM YOUTUBE COMMENTS

In [198]:
youtube = build('youtube', 'v3',developerKey=api_key)
video = youtube.commentThreads().list(part='snippet,replies',videoId='JKeRM918XDA').execute()
comments = []
while(video):
    for item in range(len(video['items'])):
        comments.append(video['items'][item]['snippet']['topLevelComment']['snippet']['textDisplay'])
    try:
        if 'nextPageToken' in video and len(comments) <1001:
            video = youtube.commentThreads().list(part = 'snippet',videoId = 'JKeRM918XDA' ,maxResults=50, pageToken=video['nextPageToken']).execute()
        else:
            break
    except:
        print("Not Much Comments")

data = ''.join(x for x in comments)


In [ ]:
data


**Above result show the commnets extracted from the youtube video**

#Cleaning data of youtube commnets 
#cleaning data with the help of package called REGULAR EXPRESSION

In [204]:
all_comments_no_urls = [ " ".join(re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', "", each_comment).split()) for each_comment in comments]

all_comments_no_urls = [ " ".join(re.sub('(@[A-Za-z0-9_]+)', "", each_comment).split()) for each_comment in comments]
all_comments_no_urls = [ " ".join(re.sub('<a.*?>|</a>', "", each_comment).split()) for each_comment in comments]
all_comments_no_urls = [ " ".join(re.sub('<br />','', each_comment).split()) for each_comment in comments]
all_comments_no_urls = [ " ".join(re.sub('</a>', '', each_comment).split()) for each_comment in comments]




# all_comments_no_urls = [ " ".join(re.sub(re.sub('</a>', '', each_comment))).split() for each_comment in comments]




CONVERTING ALL COMMENTS TO THE LOWERCASE

In [ ]:
words_in_comments = [comm.lower().split() for comm in all_comments_no_urls]
print(words_in_comments)

In [188]:

data = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                        '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', data)
data = re.sub("(@[A-Za-z0-9_]+)","", data)
data = unicodedata.normalize('NFKD', data).encode('ascii', 'ignore').decode('utf-8', 'ignore')
data = re.sub('<a.*?>|</a> ', '', data)
data = re.sub('<br />','',re.sub('</a>', '', data))


#performing the part to get the wordcloud

In [ ]:

wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="black").generate(data)
plt.figure()
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

**The above image show the wordcloud of the commnets**

#this section is counnting the words present in the previous list
#words by words

In [ ]:

# List of all words across commnet
all_words_no_urls = list(itertools.chain(*words_in_comments))

# Create counter
counts_no_urls = collections.Counter(all_words_no_urls)

counts_no_urls.most_common(12)

#Counting comments in the form of dataframe 

In [ ]:
clean_comments_no_urls = pd.DataFrame(counts_no_urls.most_common(15),
                             columns=['words', 'count'])

clean_comments_no_urls.head()

#Common words founnd in the data
#Plotting the graph with the help of bar graph

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# Plot horizontal bar graph
clean_comments_no_urls.sort_values(by='count').plot.barh(x='words',
                      y='count',
                      ax=ax,
                      color="purple")

ax.set_title("Common Words Found in comment (Including All Words)")

plt.show()

**The above graph show the word chart including all words in the  comments**

#With the help of NLTK package we are now removing the stopwords from the commnets

In [171]:
stop_words = set(stopwords.words('english'))

In [172]:

comments_with_no_sw= [[word for word in comment_words if not word in stop_words]
              for comment_words in words_in_comments]

In [ ]:
comments_with_no_sw

#list of all words with no stopwords and with there count

In [ ]:
all_words_no_sw = list(itertools.chain(*comments_with_no_sw))

counts_nsw = collections.Counter(all_words_no_sw)

counts_nsw.most_common(50)

#Now performing the SENTIMENT PART
**Sentiment like **

**(NEGATIVE,POSTIVE , NETURAL)**


#sentiment part with the help of package called TEXTBLOB

In [175]:
def get_commnet_sentiment(each_comm):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(each_comm)
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

In [ ]:
TextBlob("The video is worst").sentiment

In [177]:
sentiment_score = []
for i in range(len(comments)):
  sentiment_score.append(get_commnet_sentiment(comments[i]))


#Output of sentiment of all commnets

In [ ]:
sentiment_score

#Visualising the whole commnets data

In [179]:
scores = {'neutral':sentiment_score.count('neutral'), 'positive':sentiment_score.count('positive'), 'negative':sentiment_score.count('negative')}

In [ ]:
scores

The above output show the sentiment of the video.

In [ ]:
scores.values()

In [ ]:
scores.keys()

In [ ]:
plt.bar(scores.keys(), scores.values())
plt.show()

The above garph show the sentiment of the video with the help of graph

**This overall project show the sentiment of any youtube video with the help of python and NLTK (natrual language processing) to get the REVIEWS of any youtube video(negative,positive,neutral)**